In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import os

#import tensorflow.contrib.eager as tfe
#tf.enable_eager_execution()

In [2]:
learning_rate = 0.001
training_epochs = 21
batch_size = 100

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_basic_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## TensorBoard
log_dir = os.path.join(cur_dir, 'tensorboard', model_dir_name)
os.makedirs(log_dir, exist_ok=True)

In [4]:
mnist = keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()

In [5]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [6]:
train_x = train_x.astype(np.float32) / 255
test_x = test_x.astype(np.float32) / 255

train_x = np.reshape(train_x, [-1,28*28])
test_x = np.reshape(test_x, [-1,28*28])

train_y = to_categorical(train_y, 10) ## one-hot encoding
test_y = to_categorical(test_y,10)

# tf.data.Dataset

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=70000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(batch_size)

In [7]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 784) (60000, 10) (10000, 784) (10000, 10)


In [8]:
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
def create_model():
    def model(x):
        with tf.variable_scope('basic_model_for_mnist', reuse=False):
            layer1 = tf.layers.dense(x, 256, activation=tf.nn.sigmoid, name='layer1')
            layer2 = tf.layers.dense(layer1, 10, name='layer2')
        return layer2
    return model

def loss_fn(pred, Y):
    loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=pred, onehot_labels=Y))
    return loss

def evaluate(pred, Y):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1), tf.argmax(Y, 1)), tf.float32))
    return accuracy

pred = create_model()(X)
loss = loss_fn(pred, Y)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.1)

acc = evaluate(pred, Y)

train = optimizer.minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()

tr_x, tr_y = train_iterator.get_next()
ts_x, ts_y = test_iterator.get_next()

num_train_iter = train_x.shape[0] // batch_size
num_test_iter = test_x.shape[0] // batch_size


with tf.Session() as sess:
    sess.run(init)
    
    ## TensorBoard
    file_writer = tf.summary.FileWriter(log_dir, sess.graph)
    
    
    # restore variables
    try:
        saver.restore(sess, checkpoint_prefix+'.ckpt')
        print("Restored Variables successfully")
    except:
        print("Failed Restoring Variables")
    
    
    for epoch in range(training_epochs):
        sess.run(train_iterator.initializer)
        
        avg_loss = 0
        avg_train_acc = 0
        avg_test_acc = 0
        train_step = 0
        test_step = 0
        
        for _ in range(num_train_iter):
            x, y = sess.run([tr_x, tr_y])
            _, step_loss, step_acc = sess.run([train, loss, acc], feed_dict={X:x, Y:y})
            
            avg_loss += step_loss
            avg_train_acc += step_acc
            train_step += 1
        avg_loss = avg_loss / train_step
        avg_train_acc = avg_train_acc / train_step
        
        print("{}# epoch:".format(epoch+1))
        print("train data:")
        print("\t loss:{} \t accuracy:{}".format(avg_loss, avg_train_acc))
        
        if epoch%10 == 0:
            sess.run(test_iterator.initializer)
            for _ in range(num_test_iter):
                x, y = sess.run([ts_x, ts_y])
                step_acc = sess.run(acc, feed_dict={X:x, Y:y})
                test_step += 1
                avg_test_acc += step_acc
            avg_test_acc /= test_step
            
            print("test data:")
            print("\t accuracy:{}".format(avg_test_acc))
            saver.save(sess, checkpoint_prefix+".ckpt")
    
    file_writer.close()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\student\Git\MachineLearning\MachineLearning_basic\Edwith\Deep_learning_for_everyone_by_SKim\checkpoints\mnist_basic_seq\mnist_basic_seq.ckpt
Restored Variables successfully
1# epoch:
train data:
	 loss:0.2889839035148422 	 accuracy:0.9170333356658618
test data:
	 accuracy:0.9195000022649765
2# epoch:
train data:
	 loss:0.28778583969920873 	 accuracy:0.9170333351691564
3# epoch:
train data:
	 loss:0.28658168400327366 	 accuracy:0.917550002237161
4# epoch:
train data:
	 loss:0.2854654851680001 	 accuracy:0.917983334461848
5# epoch:
train data:
	 loss:0.2843150143697858 	 accuracy:0.9178500015536945
6# epoch:
train data:
	 loss:0.28325578474750124 	 accuracy:0.9186666695276896
7# epoch:
train data:
	 loss:0.2820936242987712 	 accuracy:0.919066669344902
8# epoch:
train data:
	 loss:0.28104534597446523 	 accuracy:0.9194666683673859
9# epoch:
train data:
	

NameError: name 'writer' is not defined

In [ ]:
!tensorboard --logdir=tensorboard\mnist_basic_seq